In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.python.keras.utils import conv_utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import pickle


from random import randint, seed
import itertools
import cv2

import math
import random
from PIL import Image, ImageDraw

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2


## Notebook 

#### Codes are mainly from : [Partial Convolution Keras](https://github.com/MathiasGruber/PConv-Keras)


#### Purpose : 

#### 1. Testing some properties about texture & content synthesis for image inpainting task

#### 2. This notebook trains 2 models to test the idea from [Instance Normalization: The Missing Ingredient for Fast Stylization](https://arxiv.org/abs/1607.08022)

#### 3. And further do some experiment about difference between In and Bn, better understanding the idea from  [A Neural Algorithm of Artistic Style](https://arxiv.org/abs/1508.06576) and [Perceptual Losses for Real-Time Style Transfer and Super-Resolution](https://arxiv.org/abs/1603.08155)

# Model

## Partial Convolution

In [2]:
class PConv2D(tf.keras.layers.Conv2D):
    def __init__(self, *args, n_channels=3, mono=False, **kwargs):
        super().__init__(*args, **kwargs)
        self.input_spec = [tf.keras.layers.InputSpec(ndim=4), tf.keras.layers.InputSpec(ndim=4)]

    def build(self, input_shape):        
        """Adapted from original _Conv() layer of Keras        
        param input_shape: list of dimensions for [img, mask]
        """
        
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
            
        if input_shape[0][channel_axis] is None:
            raise ValueError('The channel dimension of the inputs should be defined. Found `None`.')
            
        self.input_dim = input_shape[0][channel_axis]
        
        # Image kernel
        kernel_shape = self.kernel_size + (self.input_dim, self.filters)
        self.kernel = self.add_weight(shape=kernel_shape,
                                      initializer=self.kernel_initializer,
                                      name='img_kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        # Mask kernel
        self.kernel_mask = K.ones(shape=self.kernel_size + (self.input_dim, self.filters))

        # Calculate padding size to achieve zero-padding
        self.pconv_padding = (
            (int((self.kernel_size[0]-1)/2), int((self.kernel_size[0]-1)/2)), 
            (int((self.kernel_size[0]-1)/2), int((self.kernel_size[0]-1)/2)), 
        )

        # Window size - used for normalization
        self.window_size = self.kernel_size[0] * self.kernel_size[1]
        
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.filters,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.built = True

    def call(self, inputs, mask=None):

        # Padding done explicitly so that padding becomes part of the masked partial convolution
        images = K.spatial_2d_padding(inputs[0], self.pconv_padding, self.data_format)
        masks = K.spatial_2d_padding(inputs[1], self.pconv_padding, self.data_format)

        # Apply convolutions to mask
        mask_output = K.conv2d(
            masks, self.kernel_mask, 
            strides=self.strides,
            padding='valid',
            data_format=self.data_format,
            dilation_rate=self.dilation_rate
        )

        # Apply convolutions to image
        img_output = K.conv2d(
            (images*masks), self.kernel, 
            strides=self.strides,
            padding='valid',
            data_format=self.data_format,
            dilation_rate=self.dilation_rate
        )        

        # Calculate the mask ratio on each pixel in the output mask
        mask_ratio = self.window_size / (mask_output + 1e-8)

        # Clip output to be between 0 and 1
        mask_output = K.clip(mask_output, 0, 1)

        # Remove ratio values where there are holes
        mask_ratio = mask_ratio * mask_output

        # Normalize iamge output
        img_output = img_output * mask_ratio

        # Apply bias only to the image (if chosen to do so)
        if self.use_bias:
            img_output = K.bias_add(
                img_output,
                self.bias,
                data_format=self.data_format)
        
        # Apply activations on the image
        if self.activation is not None:
            img_output = self.activation(img_output)
            
        return [img_output, mask_output]
    
    def compute_output_shape(self, input_shape):
        space = input_shape[0][1:-1]
        new_space = []
        for i in range(len(space)):
            new_dim = conv_utils.conv_output_length(
                space[i],
                self.kernel_size[i],
                padding='same',
                stride=self.strides[i],
                dilation=self.dilation_rate[i])
            new_space.append(new_dim)
        new_shape = (input_shape[0][0],) + tuple(new_space) + (self.filters,)
        return [new_shape, new_shape]

## Instance Normalization

In [3]:
class InstanceNormalization(tf.keras.layers.Layer):
    def __init__(self, epsilon=1e-5):
        super(InstanceNormalization, self).__init__()
        self.epsilon = epsilon
        
    def build(self, input_shape):
        self.scale = self.add_weight(
            name='scale',
            shape=input_shape[-1:],
            initializer=tf.random_normal_initializer(1., 0.02),trainable=True)

        self.offset = self.add_weight(
            name='offset',
            shape=input_shape[-1:],
            initializer='zeros',
            trainable=True)
        
    def call(self, x):
        mean, variance = tf.nn.moments(x, axes=[1, 2], keepdims=True)
        inv = tf.math.rsqrt(variance + self.epsilon)
        normalized = (x - mean) * inv
        return self.scale * normalized + self.offset

## Generator UNET 

In [4]:
    def generator(norm_type='Bn',train=True):      

        # INPUTS
        inputs_img = tf.keras.layers.Input((None, None, 3), name='inputs_img')
        inputs_mask = tf.keras.layers.Input((None, None, 3), name='inputs_mask')
        
        
        def encoder_layer(img_in, mask_in, filters, kernel_size,norm_type,norm=True):
            conv, mask = PConv2D(filters, kernel_size, strides=2, padding='same')([img_in, mask_in])
            if norm:
                if norm_type=='In':
                    conv=InstanceNormalization()(conv)
                else:
                    conv=tf.keras.layers.BatchNormalization()(conv,training=train)
                    
            conv = tf.keras.layers.ReLU()(conv)
            return conv, mask

        
        e_conv1, e_mask1 = encoder_layer(inputs_img, inputs_mask, 64, 7, norm_type,norm=False)
        e_conv2, e_mask2 = encoder_layer(e_conv1, e_mask1, 128, 5,norm_type)
        e_conv3, e_mask3 = encoder_layer(e_conv2, e_mask2, 256, 5,norm_type)
        e_conv4, e_mask4 = encoder_layer(e_conv3, e_mask3, 512, 3,norm_type)
        e_conv5, e_mask5 = encoder_layer(e_conv4, e_mask4, 512, 3,norm_type)
        e_conv6, e_mask6 = encoder_layer(e_conv5, e_mask5, 512, 3,norm_type)
        e_conv7, e_mask7 = encoder_layer(e_conv6, e_mask6, 512, 3,norm_type)
        e_conv8, e_mask8 = encoder_layer(e_conv7, e_mask7, 512, 3,norm_type)
        
        
        def decoder_layer(img_in, mask_in, e_conv, e_mask, filters, kernel_size,norm_type, norm=True):
            up_img = tf.keras.layers.UpSampling2D(size=(2, 2))(img_in)
            up_mask = tf.keras.layers.UpSampling2D(size=(2, 2))(mask_in)
            concat_img = tf.keras.layers.Concatenate(axis=-1)([e_conv, up_img])
            concat_mask = tf.keras.layers.Concatenate(axis=-1)([e_mask, up_mask])
            
            conv, mask = PConv2D(filters, kernel_size, padding='same')([concat_img, concat_mask])
            
            if norm:
                if norm_type=='In':
                    conv=InstanceNormalization()(conv)
                else:
                    conv=tf.keras.layers.BatchNormalization()(conv,training=train)
            conv = tf.keras.layers.LeakyReLU(alpha=0.2)(conv)
            return conv, mask
            
        d_conv9, d_mask9 = decoder_layer(e_conv8, e_mask8, e_conv7, e_mask7, 512, 3,norm_type)
        d_conv10, d_mask10 = decoder_layer(d_conv9, d_mask9, e_conv6, e_mask6, 512, 3,norm_type)
        d_conv11, d_mask11 = decoder_layer(d_conv10, d_mask10, e_conv5, e_mask5, 512, 3,norm_type)
        d_conv12, d_mask12 = decoder_layer(d_conv11, d_mask11, e_conv4, e_mask4, 512, 3,norm_type)
        d_conv13, d_mask13 = decoder_layer(d_conv12, d_mask12, e_conv3, e_mask3, 256, 3,norm_type)
        d_conv14, d_mask14 = decoder_layer(d_conv13, d_mask13, e_conv2, e_mask2, 128, 3,norm_type)
        d_conv15, d_mask15 = decoder_layer(d_conv14, d_mask14, e_conv1, e_mask1, 64, 3,norm_type)
        d_conv16, d_mask16 = decoder_layer(d_conv15, d_mask15, inputs_img, inputs_mask, 3, 3, norm_type,norm=False)
        outputs = tf.keras.layers.Conv2D(3, 1, activation = 'sigmoid', name='outputs_img')(d_conv16)
        
        # Setup the model inputs / outputs
        model = tf.keras.Model(inputs=[inputs_img, inputs_mask], outputs=outputs)

        return model

## VGG Extractor

In [5]:
def VGG():

    #pool1, pool2 and pool3 for both perceptual loss & style loss
    
    vgg16=VGG16(include_top=False,weights=None)
    vgg16.load_weights('/kaggle/input/vgg16-weights/pytorch_to_keras_vgg16/pytorch_to_keras_vgg16.h5',by_name=True)
    vgg16.trainable=False
    
    layer_names=['block1_pool','block2_pool','block3_pool']
    
    outputs = [vgg16.get_layer(name).output for name in layer_names]
    
    return tf.keras.Model([vgg16.input], outputs)

# Training

## Prepare Data

In [21]:
img_size=256

batch_size=16

lr=1e-4

epochs=20

AUTOTUNE = tf.data.experimental.AUTOTUNE

fine_tune=True #fine tune
frozen_layers=[5,8,11,14,17,20,23]

In [ ]:
!mkdir ~/.kaggle
!mkdir submission
!touch ~/.kaggle/kaggle.json

api_token = {"username":"spartan1001","key":"b410e5c12a85cbb74fab8586d2e648b7"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c photo-reconstruction
!unzip ./photo-reconstruction.zip

In [8]:
!mkdir Dataset/newtrain_masked
!mkdir Dataset/newtrain_unmasked

In [9]:

animals = ['Cat','Dog','Elephant','Tiger']
pth = 'Dataset/Training_Data/Cat/Unmasked_Train/Cat-Train (1).jpeg'
new_pth_masked = 'Dataset/newtrain_masked/'
new_pth_unmasked = 'Dataset/newtrain_unmasked/'

  


# shape1 = [(158, 79), (158+75, 79+75)]
# shape2 = [(158,115),(158+75,115+75)]
# shape = [(x1,y1),(x1+75,y1+75)]

# creating new Image object
# img = Image.open(r"Cat-Train (1).jpeg")
# img = Image.new("RGB", (w, h),color = (255, 255, 255))

# create  rectangleimage


# img.show()

In [10]:
pth='Dataset/Training_Data/Cat/Unmasked_Train'
pth_mask='Dataset/Training_Data/Cat/Masked_Train'

new_pth_masked = 'Dataset/newtrain_masked'
new_pth_unmasked = 'Dataset/newtrain_unmasked'
#train
train_folder = [[],[],[],[],[]]
train_mask_folder = [[],[],[],[],[]]
w, h = 256, 256
# for i in os.listdir(f'{pth}'):
#     print(i)
animals = ['Cat','Dog','Elephant','Tiger']
i = 0
counter = 1
for animal in animals:
    pth = 'Dataset/Training_Data/'+animal+'/Unmasked_Train'
    pth_mask = 'Dataset/Training_Data/'+animal+'/Masked_Train'
    for filename in os.listdir(f'{pth}'):
        if filename[-3:]=="csv":
            print("found"+filename)
        train_folder[i].append(filename)
        x1 = random.randint(0, w-75)
        y1 = random.randint(0, h-75)
        x2 = random.randint(0, w-75)
        y2 = random.randint(0, h-75)
        
        shape1 = [( x1,y1), (x1+75, y1+75)]
        shape2 = [(x2,y2),(x2+75,y2+75)]
        
        img = Image.open(pth+"/"+filename)
        img.save(new_pth_unmasked+"/"+str(counter)+".jpeg")
        
        img1 = ImageDraw.Draw(img)  
        img1.rectangle(shape1, fill ="#000000")
        img1.rectangle(shape2,fill="#000000")
        img.save(new_pth_masked+"/"+str(counter)+".jpeg")
        
        counter = counter + 1
        
        x1 = random.randint(0, w-75)
        y1 = random.randint(0, h-75)
        x2 = random.randint(0, w-75)
        y2 = random.randint(0, h-75)
        
        shape1 = [( x1,y1), (x1+75, y1+75)]
        shape2 = [(x2,y2),(x2+75,y2+75)]
        
        img = Image.open(pth+"/"+filename)
        img.save(new_pth_unmasked+"/"+str(counter)+".jpeg")
        
        img1 = ImageDraw.Draw(img)  
        img1.rectangle(shape1, fill ="#000000")
        img1.rectangle(shape2,fill="#000000")
        img.save(new_pth_masked+"/"+str(counter)+".jpeg")
        counter = counter+1
        
    for filename in os.listdir(f'{pth_mask}'):
        if filename[-3:]=="csv":
            print("found"+filename)
            continue
        train_mask_folder[i].append(filename)
    i = i +1
for filename in os.listdir(f'{new_pth_unmasked}'):
    train_folder[i].append(filename)
    train_mask_folder[i].append(filename)
i = 0
animals = ['Cat','Dog','Elephant','Tiger','new']
for animal in animals:
    if animal=="new":
        pth = new_pth_unmasked
        pth_mask = new_pth_masked
    else:
        pth = 'Dataset/Training_Data/'+animal+'/Unmasked_Train'
        pth_mask = 'Dataset/Training_Data/'+animal+'/Masked_Train'
    if i==0:
        df_train = pd.DataFrame(np.vstack([train_folder[i],train_mask_folder[i]]).T,columns=['pth','pth_mask'])
        df_train['pth']=df_train['pth'].apply(lambda x: os.path.join(f'{pth}/{x}'))
        df_train['pth_mask']=df_train['pth_mask'].apply(lambda x: os.path.join(f'{pth_mask}/{x}'))
    else:
        temp_train = pd.DataFrame(np.vstack([train_folder[i],train_mask_folder[i]]).T,columns=['pth','pth_mask'])
        temp_train['pth']=temp_train['pth'].apply(lambda x: os.path.join(f'{pth}/{x}'))
        temp_train['pth_mask']=temp_train['pth_mask'].apply(lambda x: os.path.join(f'{pth_mask}/{x}'))
        df_train = pd.concat([df_train, temp_train], ignore_index=True)
    i = i+1

        
# for i in os.listdir(f'{pth_mask}'):
#     print(i)
#     train_folder.append(i)

#     train_folder += i
# train_folder = (os.listdir(f'{pth}'))
# print(len(train_folder[0]))
# print(len(train_mask_folder[0]))
# train_mask_folder=sorted(os.listdir(f'{pth_mask}/train'))
# df_train=pd.DataFrame(np.vstack([train_folder,train_mask_folder]).T,columns=['pth','pth_mask'])
# df_train['pth']=df_train['pth'].apply(lambda x: os.path.join(f'{pth}/train/{x}'))
# df_train['pth_mask']=df_train['pth_mask'].apply(lambda x: os.path.join(f'{pth_mask}/train/{x}'))
print(df_train)

#val
# val_folder=sorted(os.listdir(f'{pth}/val'))
# val_mask_folder=sorted(os.listdir(f'{pth_mask}/val'))
# df_val=pd.DataFrame(np.vstack([val_folder,val_mask_folder]).T,columns=['pth','pth_mask'])
# df_val['pth']=df_val['pth'].apply(lambda x: os.path.join(f'{pth}/val/{x}'))
# df_val['pth_mask']=df_val['pth_mask'].apply(lambda x: os.path.join(f'{pth_mask}/val/{x}'))

foundmasked_info.csv
foundmasked_info.csv
foundmasked_info.csv
foundmasked_info.csv
                                                     pth  \
0      Dataset/Training_Data/Cat/Unmasked_Train/Cat-T...   
1      Dataset/Training_Data/Cat/Unmasked_Train/Cat-T...   
2      Dataset/Training_Data/Cat/Unmasked_Train/Cat-T...   
3      Dataset/Training_Data/Cat/Unmasked_Train/Cat-T...   
4      Dataset/Training_Data/Cat/Unmasked_Train/Cat-T...   
...                                                  ...   
20995                Dataset/newtrain_unmasked/1987.jpeg   
20996                Dataset/newtrain_unmasked/3632.jpeg   
20997               Dataset/newtrain_unmasked/10485.jpeg   
20998                Dataset/newtrain_unmasked/4788.jpeg   
20999                Dataset/newtrain_unmasked/2108.jpeg   

                                                pth_mask  
0      Dataset/Training_Data/Cat/Masked_Train/Cat-Tra...  
1      Dataset/Training_Data/Cat/Masked_Train/Cat-Tra...  
2      Dataset/Tra

In [22]:
def get_image(path,path_mask):
    image = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
    image=tf.cast(tf.image.resize(image,(img_size,img_size)),'float32')
    image=image/255.
    
    mask = tf.image.decode_jpeg(tf.io.read_file(path_mask), channels=3)
    mask=tf.cast(tf.image.resize(mask,(img_size,img_size)),'float32')
    mask=mask/255.
    return image,mask

In [23]:
#all 0~255
ds_train=tf.data.Dataset.from_tensor_slices((df_train['pth'],df_train['pth_mask'])).map(get_image,num_parallel_calls=AUTOTUNE).\
                        shuffle(256).batch(batch_size,drop_remainder=True)

# ds_val=tf.data.Dataset.from_tensor_slices((df_val['pth'],df_val['pth_mask'])).map(get_image,num_parallel_calls=AUTOTUNE).\
#                         batch(batch_size,drop_remainder=True)

## Objective

In [24]:
vgg=VGG()


In [25]:
def l1(y_true, y_pred):
    if K.ndim(y_true) == 4:
        return K.mean(K.abs(y_pred - y_true), axis=[1,2,3])
    elif K.ndim(y_true) == 3:
        return K.mean(K.abs(y_pred - y_true), axis=[1,2])
    
    
def gram_matrix(x):
    # Permute channels and get resulting shape
    x = tf.transpose(x, perm=(0, 3, 1, 2))
    shape = tf.shape(x)
    B, C, H, W = shape[0], shape[1], shape[2], shape[3]
        
    # Reshape x and do batch dot product
    features = tf.reshape(x, tf.stack([B, C, H*W]))
    gram = tf.keras.backend.batch_dot(features, features, axes=2)
        
    # Normalize with channels, height and width
    gram = gram /  tf.cast(C * H * W, x.dtype)
        
    return gram

In [26]:

def loss_hole(mask, y_true, y_pred):
    return l1((1-mask) * y_true, (1-mask) * y_pred)
    
def loss_valid(mask, y_true, y_pred):
    return l1(mask * y_true, mask * y_pred)
    
def loss_perceptual(vgg_out, vgg_gt, vgg_comp): 
    loss = 0
    for o, c, g in zip(vgg_out, vgg_comp, vgg_gt):
        loss += l1(o, g) + l1(c, g)
    return loss
        
def loss_style(output, vgg_gt):
    loss = 0
    for o, g in zip(output, vgg_gt):
        loss += l1(gram_matrix(o), gram_matrix(g))
    return loss

    
def loss_tv(mask, y_comp):
    kernel = tf.ones(shape=(3, 3, mask.shape[3], mask.shape[3]))
    dilated_mask = K.conv2d(1-mask, kernel, data_format='channels_last', padding='same')

    dilated_mask = tf.cast(K.greater(dilated_mask, 0), 'float32')
    P = dilated_mask * y_comp

    a = l1(P[:,1:,:,:], P[:,:-1,:,:])
    b = l1(P[:,:,1:,:], P[:,:,:-1,:])        
    return a+b

In [27]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

def loss_func(y_true, y_pred, mask):
    y_comp = mask * y_true + (1-mask) * y_pred
    vgg_out = vgg((y_pred-mean)/std)
    vgg_gt = vgg((y_true-mean)/std)
    vgg_comp = vgg((y_comp-mean)/std)
            
    l1 = loss_valid(mask, y_true, y_pred)
    l2 = loss_hole(mask, y_true, y_pred)
    l3 = loss_perceptual(vgg_out, vgg_gt, vgg_comp)
    l4 = loss_style(vgg_out, vgg_gt)
    l5 = loss_style(vgg_comp, vgg_gt)
    l6 = loss_tv(mask, y_comp)
    
    return l1 + 6*l2 + 0.05*l3 + 120*(l4+l5) + 0.1*l6

## Train

In [28]:
def show(x,mask,model,n=6):
    
    x_masked= x*mask+(1-mask)
    
    x_pred=model([x_masked,mask],training=False)
    
    mask = tf.concat([mask for _ in range(3)], -1)
    
    fig,ax=plt.subplots(nrows=3,ncols=n,figsize=(8,8))
    
    for i in range(3):
        for j in range(n):
            if i==1:
                x=x_masked
            elif i==2:
                x=x_pred
            ax[i,j].imshow(x[j])
    plt.show()

In [29]:
@tf.function
def train_step(x,mask,model,opt):
    with tf.GradientTape() as tape:
        x_masked= x*mask+(1-mask)
        
        x_prime=model([x,mask],training=True)
        
        loss=tf.reduce_mean(loss_func(x,x_prime,mask))
        
    grad=tape.gradient(loss,model.trainable_variables)
    opt.apply_gradients(zip(grad,model.trainable_variables))
    
    return loss

In [30]:
def train(learning_rate = 1e-5):
    try:
        G=tf.keras.models.load_model('../input/inpainting-models/model_bn/model_in')
        
        if fine_tune:
            for i in frozen_layers:
                G.layers[i].trainable=False
                lr=1e-5
    except:
        G=generator('In')
    lr = learning_rate
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr)
    ckpt = tf.train.Checkpoint(G=G,optimizer=optimizer)
    ckpt_manager = tf.train.CheckpointManager(ckpt,'./ckpt', max_to_keep=1)
    print('start training')
    for epoch in range(epochs):
        if epoch%5==0:
            print('sampling')
#             for x,mask in ds_val:
#                 show(x,mask,G)
#                 break 
                
            #save
            ckpt_manager.save()
            tf.keras.models.save_model(G,'./model')
            
        loop=tqdm(ds_train)
        for x,mask in loop:
            loss=train_step(x,mask,G,optimizer)
            loop.set_postfix(loss=f'loss:{loss}')
    return G

In [31]:
 
G=train()

start training
sampling


  2%|▏         | 26/1312 [00:40<33:45,  1.58s/it, loss=loss:8.809263229370117]  


KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

img_path = 'Dataset/Testing_Data/Cat-Train (1).png'  # path to test image
img = cv2.imread(img_path)  # load image using OpenCV
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert to RGB format

# Preprocess the image by normalizing pixel values and resizing
img = cv2.resize(img, (256, 256))  # resize to match model input shape
img = img / 255.0  # normalize pixel values to range [0, 1]
img = np.expand_dims(img, axis=0)  # add batch dimension
model = G
# Use the model to generate the inpainted image
inpainted_img = model.predict([img,img])
# Convert the inpainted image from tensor to numpy array
inpainted_img = inpainted_img[0]  # remove batch dimension
print(inpainted_img[27][0][1])

# inpainted_img = inpainted_img * 255.0  # rescale pixel values to range [0, 255]
# inpainted_img = inpainted_img.astype('uint8')  # convert to integer type
# inpainted_img = cv2.cvtColor(inpainted_img, cv2.COLOR_RGB2BGR)  # convert to BGR format

# Display the original and inpainted images side by side
# fig, ax = plt.subplots(1, 2, figsize=(10, 5))
# ax[0].imshow(cv2.cvtColor(img[0], cv2.COLOR_RGB2BGR))
# ax[0].set_title('Original Image')
# ax[1].imshow(inpainted_img)
# ax[1].set_title('Inpainted Image')
# plt.show()

In [ ]:
content_dir = "Dataset/Testing_Data/"
mask_dir = "/kaggle/working/mask_files"
gen_dir = "/kaggle/working/gen_files"
sub_dir = "submission"
metadata = pd.read_csv(content_dir + "masked_info.csv")
f = open("Final_csv_file.csv","w")
f.write("filename_box_pixel,Value\n")
# f = open(sub_dir+"/DL_Group28_Sub1.csv","w")
vals_dict = {
    'filename_box_pixel' : [],
    'Value' : []
}
print('meta'data.shape[0])


# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert to RGB format


model = G

# print(inpainted_img)

for idx in range(metadata.shape[0]):
    _, filename, mask1_y, mask1_x, mask2_y, mask2_x = metadata.iloc[idx,:].tolist()
    cords_box1 = [(row, col) for row in range(mask1_y,  mask1_y + 75) for col in range(mask1_x,  mask1_x + 75)]
    cords_box2 = [(row, col) for row in range(mask2_y,  mask2_y + 75) for col in range(mask2_x,  mask2_x + 75)]
    box_cords = {'box1' : cords_box1, 'box2' : cords_box2}

#     gen_filename = gen_dir + "/" + filename[0:filename.find(".")] + '.jpg'

#     gen_img = Image.open(gen_filename)
    img_path = 'Dataset/Testing_Data/'+filename  # path to test image
    img = cv2.imread(img_path)  # load image using OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # convert to RGB format
    
        # Preprocess the image by normalizing pixel values and resizing
    img = cv2.resize(img, (256, 256))  # resize to match model input shape
    img = img / 255.0  # normalize pixel values to range [0, 1]
    img = np.expand_dims(img, axis=0)  # add batch dimension
        # Use the model to generate the inpainted image
    inpainted_img = model.predict([img,img])
    # Convert the inpainted image from tensor to numpy array
    test_results = inpainted_img[0]  # remove batch dimension
    for box, all_cords in box_cords.items():
        for px_cords in all_cords:
            b = test_results[px_cords[0]][px_cords[1]][0]
            g = test_results[px_cords[0]][px_cords[1]][1]
            r = test_results[px_cords[0]][px_cords[1]][2]
#             r, g, b = gen_img.getpixel((px_cords[0], px_cords[1]))
#             print(b)
            px_name_b = filename + "_" + box + "_" + str(px_cords[0]) + "_" + str(px_cords[1]) + "_0"
            px_name_g = filename + "_" + box + "_" + str(px_cords[0]) + "_" + str(px_cords[1]) + "_1"
            px_name_r = filename + "_" + box + "_" + str(px_cords[0]) + "_" + str(px_cords[1]) + "_2"
#             print(px_name_b+","+str(b))
#             b = (b+1)/2
#             g = (g+1)/2
#             r = (r+1)/2
            f.write(px_name_b+","+str(b)+"\n")
            f.write(px_name_g+","+str(g)+"\n")
            f.write(px_name_r+","+str(r)+"\n")
#             vals_dict['filename_box_pixel'].extend([px_name_b, px_name_g, px_name_r])
#             vals_dict['Value'].extend([b, g, r])

    if idx % 5 == 0:
        print(idx)
f.close()
# print('Lists Created')
# sub_df = pd.DataFrame(vals_dict)
# print('DF Created')
# # sub_df['Value'] = sub_df['Value'] / 255
# # print('Score Normalised')
# sub_df.to_csv(sub_dir+"/"'DL_Group28_Sub1.csv', index=False)
# print('CSV Generated')